## Iteration 3: includes four type of stroke patients and uses in the  AcuteStrokeUnit class to generate arrivals for each type of patient.

## **. Patient Admission Sources**  
Patients enter the hospital through **two primary pathways**:  
- **New Admissions**: Direct hospital entry for Stroke, TIA, Complex Neurological, and Other Medical Cases.  


| Patient Type                     | Admissions (n) | Percentage (%) |
|----------------------------------|---------------|--------------|
| Stroke                           | 1,320         | 54%          |
| Transient Ischemic Attack (TIA)  | 158           | 6%           |
| Complex Neurological             | 456           | 19%          |
| Other Medical Cases              | 510           | 21%          |

**Patient admissions are distributed as follows**

| Category                | Mean(Days) |
|-------------------------|----------------------------|
| Stroke                  | 1.2                        |
| TIA (Transient Ischemic Attack) | 9.3                |
| Complex Neurological    | 3.6                        |
| Other                   | 3.2                        |



## 1. Imports 

In [4]:
import numpy as np
import itertools
import simpy
import os
import sys

In [5]:
# Add parent directory to path so we can import distribution.py
sys.path.append(os.path.abspath('..'))  # noqa: E402

from distribution import Exponential  # noqa: E402

## 2. Constants

In [6]:
# default mean inter-arrival times(exp)
IAT_STROKE = 1.2
IAT_TIA = 9.3
IAT_COMPLEX_NEURO = 3.6
IAT_OTHER = 3.2

# sampling settings
N_STREAMS = 4
DEFAULT_RND_SET = 0

# Boolean switch to simulation results as the model runs
TRACE = False

# run variables (units = hours)
RUN_LENGTH = 24 * 10

## 2. Helper classes and functions

In [7]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

## 3. Experiment class

In [8]:
class Experiment:
    """
    Encapsulates the concept of an experiment
    for the Acute Stroke Unit simulation.
    Manages parameters, PRNG streams, and results.
    """

    def __init__(
        self,
        random_number_set=0,
        n_streams=4,
        iat_stroke=1.2,
        iat_tia=9.3,
        iat_complex_neuro=3.6,
        iat_other=3.2,
        asu_bed=10,
    ):
        """
        Initialize default parameters.
        """
        # Sampling settings
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # Model parameters
        self.iat_stroke = iat_stroke
        self.iat_tia = iat_tia
        self.iat_complex_neuro = iat_complex_neuro
        self.iat_other = iat_other
        self.asu_bed = asu_bed

        # Length of stay distributions
        self.length_of_stay = {
            "stroke": (7.4, 8.6),
            "tia": (1.8, 2.3),
            "complex_neuro": (4.0, 5.0),
            "other": (3.8, 5.2),
        }

        # Initialize results storage
        self.init_results_variables()

        # Initialize sampling distributions
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling.

        Parameters:
        ----------
        random_number_set: int
            Controls the set of pseudo-random numbers
            used by the distributions.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Creates the distributions used by the model and initializes
        the random seeds of each.
        """
        # Produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # Inter-arrival time distributions
        self.arrival_stroke = Exponential(
            self.iat_stroke, random_seed=self.seeds[0]
        )
        self.arrival_tia = Exponential(self.iat_tia, random_seed=self.seeds[1])
        self.arrival_complex_neuro = Exponential(
            self.iat_complex_neuro, random_seed=self.seeds[2]
        )
        self.arrival_other = Exponential(
            self.iat_other, random_seed=self.seeds[3]
        )

    def init_results_variables(self):
        """
        Initializes all the experiment variables used in results collection.
        """
        self.results = {}
        self.results["n_stroke"] = 0
        self.results["n_tia"] = 0
        self.results["n_complex_neuro"] = 0
        self.results["n_other"] = 0
        self.results["n_patients"] = 0

## 3. Patient Class

In [9]:
class Patient:
    """
    Represents a patient in the system.
    """

    def __init__(self, patient_id, env, args, acute_stroke_unit):
        self.patient_id = patient_id
        self.env = env
        self.args = args
        self.acute_stroke_unit = acute_stroke_unit  # Pass the ASU instance

    def treatment(self):
        """
        Simulates patient treatment process.
        """
        arrival_time = self.env.now
        with self.acute_stroke_unit.asu_bed.request() as request:
            yield request
            waiting_time = self.env.now - arrival_time
            trace(
                f"Patient {self.patient_id}"
                f" gets a bed at {self.env.now:.2f} days "
                f" waiting {waiting_time:.2f} days"

            )

### 4. Acute Stroke Unit Class


In [10]:
class AcuteStrokeUnit:
    """
    Models the Acute Stroke Unit (ASU) in the hospital.

    Parameters:
    - env: SimPy environment
    - args: Scenario object containing hospital parameters
    """

    def __init__(self, env, args):
        self.env = env
        self.patient_count = 0
        self.args = args
        self.asu_bed = simpy.Resource(env, capacity=args.asu_bed)

    def stroke_arrivals_generator(self):
        """
        Arrival process for stroke patients.
        """
        for patient_count in itertools.count(start=1):
            inter_arrival_time = self.args.arrival_stroke.sample()
            yield self.env.timeout(inter_arrival_time)

            self.args.results["n_stroke"] = patient_count
            self.args.results["n_patients"] += 1
            trace(f"{self.env.now:.2f}: STROKE arrival.")

            new_patient = Patient(patient_count, self.env, self.args, self)
            self.env.process(new_patient.treatment())

    def tia_arrivals_generator(self):
        """
        Arrival process for TIA patients.
        """
        for patient_count in itertools.count(start=1):
            inter_arrival_time = self.args.arrival_tia.sample()
            yield self.env.timeout(inter_arrival_time)

            self.args.results["n_tia"] = patient_count
            self.args.results["n_patients"] += 1
            trace(f"{self.env.now:.2f}: TIA arrival.")

            new_patient = Patient(patient_count, self.env, self.args, self)
            self.env.process(new_patient.treatment())

    def complex_neuro_arrivals_generator(self):
        """
        Arrival process for complex neuro cases.
        """
        for patient_count in itertools.count(start=1):
            inter_arrival_time = self.args.arrival_complex_neuro.sample()
            yield self.env.timeout(inter_arrival_time)

            self.args.results["n_complex_neuro"] = patient_count
            self.args.results["n_patients"] += 1
            trace(f"{self.env.now:.2f}: COMPLEX NEURO arrival.")

            new_patient = Patient(patient_count, self.env, self.args, self)
            self.env.process(new_patient.treatment())

    def other_arrivals_generator(self):
        """
        Arrival process for other neurological cases.
        """
        for patient_count in itertools.count(start=1):
            inter_arrival_time = self.args.arrival_other.sample()
            yield self.env.timeout(inter_arrival_time)

            self.args.results["n_other"] = patient_count
            self.args.results["n_patients"] += 1
            trace(f"{self.env.now:.2f}: OTHER arrival.")

            new_patient = Patient(patient_count, self.env, self.args, self)
            self.env.process(new_patient.treatment())

## 4. A function per arrival source

The first approach we will use is creating an arrival generator per source.  There will be some code redundancy, but it will a clear design for others to understand.

## 5. Single run function

In [11]:
def single_run(experiment, rep=0, run_length=RUN_LENGTH):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/parameters to use with model

    rep: int
        The replication number.

    run_length: float, optional (default=RUN_LENGTH)
        The run length of the model
    """

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()
    asu = AcuteStrokeUnit(env, experiment)  # Create ASU instance

    # we pass all arrival generators to simpy
    env.process(asu.stroke_arrivals_generator())
    env.process(asu.tia_arrivals_generator())
    env.process(asu.complex_neuro_arrivals_generator())
    env.process(asu.other_arrivals_generator())

    # run for warm-up + results collection period
    env.run(until=run_length)

    # return the count of the arrivals
    return experiment.results

In [12]:
TRACE = True
experiment = Experiment()
results = single_run(experiment)
results

3.95: STROKE arrival.
Patient 1 gets a bed at 3.95 days  waiting 0.00 days
4.64: OTHER arrival.
Patient 1 gets a bed at 4.64 days  waiting 0.00 days
4.87: STROKE arrival.
Patient 2 gets a bed at 4.87 days  waiting 0.00 days
4.95: OTHER arrival.
Patient 2 gets a bed at 4.95 days  waiting 0.00 days
5.51: COMPLEX NEURO arrival.
Patient 1 gets a bed at 5.51 days  waiting 0.00 days
5.92: COMPLEX NEURO arrival.
Patient 2 gets a bed at 5.92 days  waiting 0.00 days
6.37: STROKE arrival.
Patient 3 gets a bed at 6.37 days  waiting 0.00 days
6.64: STROKE arrival.
Patient 4 gets a bed at 6.64 days  waiting 0.00 days
6.91: STROKE arrival.
Patient 5 gets a bed at 6.91 days  waiting 0.00 days
7.47: TIA arrival.
Patient 1 gets a bed at 7.47 days  waiting 0.00 days
9.07: STROKE arrival.
Patient 6 gets a bed at 9.07 days  waiting 0.00 days
9.12: OTHER arrival.
Patient 3 gets a bed at 9.12 days  waiting 0.00 days
9.15: STROKE arrival.
Patient 7 gets a bed at 9.15 days  waiting 0.00 days
9.34: OTHER arriv

{'n_stroke': 200,
 'n_tia': 19,
 'n_complex_neuro': 60,
 'n_other': 80,
 'n_patients': 359}

In [13]:
# percentage of patients with stroke who are admitted to the stroke unit
stroke_percentage = results["n_stroke"] / results["n_patients"]
tia_percentage = results["n_tia"] / results["n_patients"]
complex_percentage = results["n_complex_neuro"] / results["n_patients"]
other_percentage = results["n_other"] / results["n_patients"]

print(f"% stroke: {stroke_percentage*100:.2f}%")
print(f"% tia : {tia_percentage*100:.2f}%")
print(f"% complex neuro: {complex_percentage*100:.2f}%")
print(f"% Others: {other_percentage*100:.2f}%")

% stroke: 55.71%
% tia : 5.29%
% complex neuro: 16.71%
% Others: 22.28%


## A hospital that only provides surgery for hip fractures

In [14]:
M = 1_000_000
experiment = Experiment(iat_stroke=M, iat_complex_neuro=M, iat_other=M)
results = single_run(experiment)
results

7.47: TIA arrival.
Patient 1 gets a bed at 7.47 days  waiting 0.00 days
11.02: TIA arrival.
Patient 2 gets a bed at 11.02 days  waiting 0.00 days
14.41: TIA arrival.
Patient 3 gets a bed at 14.41 days  waiting 0.00 days
19.33: TIA arrival.
Patient 4 gets a bed at 19.33 days  waiting 0.00 days
45.26: TIA arrival.
Patient 5 gets a bed at 45.26 days  waiting 0.00 days
63.50: TIA arrival.
Patient 6 gets a bed at 63.50 days  waiting 0.00 days
72.33: TIA arrival.
Patient 7 gets a bed at 72.33 days  waiting 0.00 days
111.08: TIA arrival.
Patient 8 gets a bed at 111.08 days  waiting 0.00 days
116.54: TIA arrival.
Patient 9 gets a bed at 116.54 days  waiting 0.00 days
142.47: TIA arrival.
Patient 10 gets a bed at 142.47 days  waiting 0.00 days
165.64: TIA arrival.
Patient 11 gets a bed at 165.64 days  waiting 0.00 days
182.45: TIA arrival.
Patient 12 gets a bed at 182.45 days  waiting 0.00 days
185.34: TIA arrival.
Patient 13 gets a bed at 185.34 days  waiting 0.00 days
210.21: TIA arrival.
Pat

{'n_stroke': 0,
 'n_tia': 19,
 'n_complex_neuro': 0,
 'n_other': 0,
 'n_patients': 19}